# Lab 8: Greedy Algorithms

Lab associated with Module 8: Greedy Algorithms

***

In [7]:
# The following lines are used to increase the width of cells to utilize more space on the screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

/var/folders/z_/xphnyhxs03sg7p8v5dgkr10w0000gn/T/ipykernel_17605/1945220536.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


***

### Section 0: Imports

In [8]:
import numpy as np

In [9]:
import math

In [10]:
from IPython.display import Image
from graphviz import Digraph

***

### <font color='red'> Activity 1a: Write a greedy algorithm for Activity Selection problem in this week's lab. </font>

In [2]:
#### TODO ####
### Good Luck ###
def greedyActivitySelection(activities):
    
    n = len(activities)
    A = [activities[0]]
    k = 1
    for m in range(2, n):
        
        if activities[m][0] >= activities[k][1]:
            A = A + [activities[m]]
            k = m
    return A



activities = [  [1,4],[2,5],[3,6],[5,7],[3,8],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]
activities = sorted(activities, key=lambda x: x[1])


activityList = greedyActivitySelection(activities)

print("Solution:")
for act in activityList:
    print(act)


### <font color='red'> Activity 1b: Devise a dynamic programming solution to Activity Selection problem, and see if you get the same results as that of greedy strategy. </font>

In [1]:
#### TODO ####
### Good Luck ###
def dynamicProgrammingActivitySelection(activities):
    
    maxActivities = []
    
    for k in range(0, len(activities)):
        Sij = [] # Combined list of activities from Sik, activity k, and Skj.
        Sik = [] # List of activities that finish before activity k start 
        Skj = [] # List of activities that start after activity k finish
        
        for i in range(0, len(activities)):
            if activities[k][0] > activities[i][1]: # finishes before k starts
                Sik = Sik + [activities[i]]
            elif activities[k][1] < activities[i][0]: # starts after k finishes
                Skj = Skj + [activities[i]]
        
        # The function calls itself recursively for Sik and Skj
        Sij = dynamicProgrammingActivitySelection(Sik) + [activities[k]] + dynamicProgrammingActivitySelection(Skj)
        
        if len(Sij) > len(maxActivities):
            maxActivities = Sij
    
    return maxActivities


def dynamicProgrammingActivitySelection(activities):
    # Sort activities by their finish times in ascending order
    sorted_activities = sorted(activities, key=lambda x: x[1])

    n = len(sorted_activities)
    # Initialize a table to store the maximum number of activities for each ending time
    max_activities = [1] * n

    # Build the max_activities table using dynamic programming
    for i in range(1, n):
        for j in range(i):
            if sorted_activities[i][0] >= sorted_activities[j][1] and max_activities[i] < max_activities[j] + 1:
                max_activities[i] = max_activities[j] + 1

    max_activity_count = max(max_activities)

    # Reconstruct the solution
    selected_activities = []
    idx = max_activities.index(max_activity_count)

    while max_activity_count > 1:
        for j in range(idx, -1, -1):
            if max_activities[j] == max_activity_count - 1 and sorted_activities[j][1] <= sorted_activities[idx][0]:
                selected_activities.insert(0, sorted_activities[j])
                idx = j
                max_activity_count -= 1
                break

    return selected_activities

In [ ]:
activities = [  [1,4],[2,5],[3,6],[5,7],[3,8],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]

activityList = dynamicProgrammingActivitySelection(activities)

print("Solution:")
for act in activityList:
    print(activities[act])

In [ ]:
#### TODO ####
### Good Luck ###

def dynamicProgrammingActivityRecursive(activities: list) -> list: 
    if len(activities) == 0: # if no activities in list, return empty list 
        return [] 
    activitiesList = [] # creates empty activity list
    i = 0 
    j = len(activities) - 1 
    for k in range(i+1, j): # loop through all activities in list between i and j
        if (activities[k][0] < (activities[i][0] + activities[i][1])) or (activities[k][0] + activities[k][1] > activities[j][0]):
            continue
        k_start = i
        k_end = j
        for x in range(i+1, k): # loop through all activities to find to last activity that starts before k 
            if (activities[x][0] + activities[x][1]) < activities[k][0]: 
                k_start = x
            elif (activities[x][0] + activities[x][1]) >= activities[k][0]: 
                break
        for x in range(k+1, j): # loop through all activities after k starts to find first to begin after k ends 
            if activities[x][0] > (activities[k][0] + activities[k][1]): 
                k_end = x
                break
        start = dynamicProgrammingActivityRecursive(activities[i:k_start]) # decide on activities between i and k 
        finish = dynamicProgrammingActivityRecursive(activities[k_end:j]) # decide on activities between k and j 
        lst = [activities[i]] + start + [activities[k]] + finish + [activities[j]]
        activitiesList.append(lst)
    maximum = 0
    for i in range(1,len(activitiesList)): 
        if len(activitiesList[i]) > len(activitiesList[maximum]): 
            maximum = i 
    return activitiesList[maximum]

def dynamicProgrammingActivitySelection(activities: list) -> list: 
    activities.sort()
    return dynamicProgrammingActivityRecursive(activities)


activities = [  [1,4],[3,8],[2,5],[3,6],[5,7],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]

activityList = greedyActivitySelection(activities)

print("Solution:")
for act in activityList:
    print(act)

***

###  Prim's Algorithm

Graph's Preliminaries

In [25]:
class Node:
    
    def __init__(self, v):

        self.value = v
        self.inNeighbors = []
        self.outNeighbors = []
        
        self.status = "unvisited"
        self.estD = np.inf

        self.parent = None
        
    def hasOutNeighbor(self, v):
        
        if v in self.outNeighbors:
            return True
        
        return False
        
    def hasInNeighbor(self, v):
        
        if v in self.inNeighbors:
            return True
        
        return False
    
    def hasNeighbor(self, v):
        
        if v in self.inNeighbors or v in self.outNeighbors:
            return True
        
        return False
    
    def getOutNeighbors(self):
        
        return self.outNeighbors
    
    def getInNeighbors(self):
        
        return self.inNeighbors
    
    def getOutNeighborsWithWeights(self):
        
        return self.outNeighbors
    
    def getInNeighborsWithWeights(self):
        
        return self.inNeighbors
    
    # ------------------------------------------------
    # Let us modify following two functions to incorporate weights
    # ------------------------------------------------
    
    def addOutNeighbor(self,v,wt):
        
        self.outNeighbors.append((v,wt))
    
    def addInNeighbor(self,v,wt):
        
        self.inNeighbors.append((v,wt))
    def __lt__(self, other):
        return self.value < other.value
        
        
    def __str__(self):
        
        return str(self.value) 


class Graph:
    
    def __init__(self):
        
        self.vertices = []

    def addVertex(self,n):
        
        self.vertices.append(n)
        
    # ------------------------------------------------
    # Let us modify following two functions to incorporate weights
    # ------------------------------------------------
        
    def addDiEdge(self, u, v, wt = 1):
        
        u.addOutNeighbor(v, wt = wt)
        v.addInNeighbor(u, wt = wt)
        
    # add edges in both directions between u and v
    def addBiEdge(self, u, v, wt = 1):
        
        self.addDiEdge(u, v, wt = wt)
        self.addDiEdge(v, u, wt = wt)
        
            
    # get a list of all the directed edges
    # directed edges are a list of two vertices
    def getDirEdges(self):
        
        ret = []
        for v in self.vertices:
            ret += [ [v, u] for u in v.outNeighbors ]
        return ret
    
    # reverse the edge between u and v.  Multiple edges are not supported.
    def reverseEdge(self,u,v):
        
        if u.hasOutNeighbor(v) and v.hasInNeighbor(u):
            
            if v.hasOutNeighbor(u) and u.hasInNeighbor(v): 
                return
        
            self.addDiEdge(v, u)
            u.outNeighbors.remove(v)
            v.inNeighbors.remove(u)        
                
    def __str__(self):
        
        ret = "Graph with:\n"
        ret += "\t Vertices:\n\t"
        for v in self.vertices:
            ret += str(v) + ","
        ret += "\n"
        ret += "\t Edges:\n\t"
        for a,b in self.getDirEdges():
            ret += "(" + str(a) + "," + str(b[0]) + "," + str (b[1]) + ") "
        ret += "\n"
        return ret

In [26]:
G = Graph()

for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    G.addVertex( Node(i) )

In [27]:
V = G.vertices

#0, 1, 2, 3, 4, 5, 6, 7, 8
#A, B, C, D, E, F, G, H, I

G.addBiEdge( V[0], V[1], 4)

G.addBiEdge( V[0], V[7] , 8)

G.addBiEdge( V[1], V[7], 11)

G.addBiEdge( V[1], V[2], 8)

G.addBiEdge( V[2], V[3], 7)

G.addBiEdge( V[3], V[4], 9)

G.addBiEdge( V[3], V[5], 14 )

G.addBiEdge( V[4], V[5], 10 )

G.addBiEdge( V[2], V[5], 4 )

G.addBiEdge( V[2], V[8], 2 )

G.addBiEdge( V[5], V[6], 2 )

G.addBiEdge( V[6], V[7], 1 )

G.addBiEdge( V[6], V[8], 6 )

G.addBiEdge( V[7], V[8], 7 )


In [28]:
print(G)

Graph with:
	 Vertices:
	A,B,C,D,E,F,G,H,I,
	 Edges:
	(A,B,4) (A,H,8) (B,A,4) (B,H,11) (B,C,8) (C,B,8) (C,D,7) (C,F,4) (C,I,2) (D,C,7) (D,E,9) (D,F,14) (E,D,9) (E,F,10) (F,D,14) (F,E,10) (F,C,4) (F,G,2) (G,F,2) (G,H,1) (G,I,6) (H,A,8) (H,B,11) (H,G,1) (H,I,7) (I,C,2) (I,G,6) (I,H,7) 



This is what we had in the lectures as the slow implementation of Prim's Algorithm

In [29]:
# G is graph
# s is the node to start

def slowPrim(G, s):
    
    # first, find the lightest edge leaving s
    bestWt = np.inf
    bestu = None
    
    for u,wt in s.getOutNeighborsWithWeights():
        
        if wt < bestWt:
            bestWt = wt
            bestu = u
    
    MST = [ (s, bestu) ]
    verticesVisited = [s,bestu]
    
    while len(verticesVisited) < len(G.vertices): # danger! this will loop forever if the graph isn't connected...
        
        # find the lightest edge (x,v) so that x has been visited and v hasn't.
        # Finding the Lightest Edge Connecting the MST to an Unvisited Node
        bestWt = np.inf
        bestv = None
        bestx = None

        
        for x in verticesVisited:
            for v,wt in x.getOutNeighborsWithWeights():
                
                if v in verticesVisited:
                    continue
                
                if wt < bestWt:
                    bestWt = wt
                    bestv = v
                    bestx = x
                    
        MST.append((bestx,bestv))
        verticesVisited.append(bestv)
    
    return MST

In [30]:
T = slowPrim(G, G.vertices[0])

for x,y in T:
    print(x,y)

A B
A H
H G
G F
F C
C I
C D
D E


Okay, it seems to be working fine, but as we discussed, will be quite slow. Let us see if we can work on the faster version of the code as:

### <font color='red'> Activity 2: In lights of Prim's Algorithm above, write an efficient implementation based on our discussions in the Seminar/Lecture. </font>

Why the prim implementation is more efficient?


In [31]:
def prim(G,w):
    
    for v in G.vertices:
        v.status = "unvisited"
        v.estD = np.inf
        v.parent = None
        
    w.estD = 0
    
    MST = []
    verticesVisited = []
    
    while len(verticesVisited) < len(G.vertices): # Until all the vertex are reached
        # Find the vertice u with smallest key 
        bestWt = np.inf
        bestu = None
        for u in G.vertices:
            if u.status == "unvisited" and u.estD < bestWt:
                bestWt = u.estD
                bestu = u
                
        # Update keys of u's neighbors v
        for v, wt in bestu.getOutNeighborsWithWeights():
            if wt < v.estD:
                v.estD = wt
                v.parent = bestu
        
        # Mark u as reached
        bestu.status = "visited"
        verticesVisited.append(bestu)
        
        # Add (parent[u], u) to the MST
        MST.append((bestu.parent, bestu))
                    
    return MST

In [34]:
import heapq
class Vertex:
    def __init__(self, name):
        self.name = name
        self.neighbors = []  
    
    def add_neighbor(self, neighbor, weight):
        self.neighbors.append((neighbor, weight))

# def optimized_prim(vertices, start):
#     verticesVisited = set()
#     heap = []  
#     MST = []
    
#     heapq.heappush(heap, (0, None, start))

#     while heap:
#         edge_wt, current, neighbor = heapq.heappop(heap)

#         if neighbor in verticesVisited:
#             continue
#         verticesVisited.add(neighbor)
#         if current is not None:
#             MST.append((current, neighbor))
#         for next_neighbor, next_edge_wt in neighbor.neighbors:
#             if next_neighbor not in verticesVisited:
#                 heapq.heappush(heap, (next_edge_wt, neighbor, next_neighbor))
#     return MST

# import heapq

def optimized_prim(vertices, start):
    verticesVisited = set()
    heap = []  
    MST = []
    
    heapq.heappush(heap, (0, None, start))

    while heap:
        edge_wt, current, neighbor = heapq.heappop(heap)

        if neighbor in verticesVisited:
            continue
        verticesVisited.add(neighbor)
        if current is not None:
            MST.append((current, neighbor))
        for next_neighbor, next_edge_wt in neighbor.getOutNeighborsWithWeights():
            if next_neighbor not in verticesVisited:

                heapq.heappush(heap, (next_edge_wt, neighbor, next_neighbor))
    return MST



In [33]:
T = optimized_prim(G, G.vertices[0])


for x,y in T:
    print(x,y)

4
8
11
8
1
7
2
6
14
10
4
7
2
9
A B
A H
H G
G F
F C
C I
C D
D E


***